In [ ]:
ls

2017-fordgobike-tripdataa.csv  sample_data/


In [ ]:
pwd

'/content'

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!ls

2017-fordgobike-tripdataa.csv  spark-3.1.1-bin-hadoop3.2
sample_data		       spark-3.1.1-bin-hadoop3.2.tgz


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Google Colab - PySpark Demo") \
        .master("local[*]") \
        .getOrCreate()


In [ ]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from haversine import haversine
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from geopy.distance import geodesic
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import pyspark.sql.functions as F

# Tasks
## Week 1
###1 - open databricks account  - v
###2 - clean the dataset - v
###3 - calculate distance of each trip using haversine library and add the result to the dataset - v

##Week 2
###1 - calculate the duration in seconds of each trip - x
###2 - by assuming each minute cost 0.35 cent calculate the fee for each trip
###3 - calculate the total distance for each bike and list the top 10

##Week 3
###1 - calculate the number of trips for each start station list top 10 and find the ratio of using as male or female
###2 - make a comparison to find the percentage of usage for customer and subscriber
###3 - calculate the age of all users and show the relation between the distance and the age
###4 - prepare for you presentation

##Week 4
###1 - calculate the total cost for all customers and all subscribers
###2 - Analyze how often bikes are being rented ( morning - afternoon - evening) and and find the density for each hour of the day
###3 - find if there is a relation between the time and start stations

##Week 5
###1 - what is the ratio from credit card or app wallet


# Week 1

task 3 - calculate distance of each trip using haversine library and add the result to the dataset

In [ ]:
df = spark.read.csv('2017-fordgobike-tripdataa.csv', inferSchema = True, header = True)


In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    start = (lat1, lon1)
    end = (lat2, lon2)
    distance = haversine(start, end)
    return distance

calculate_distance = udf(haversine_distance, DoubleType())

# Apply the user-defined function to the DataFrame
df = df.withColumn("distance", calculate_distance(df.start_station_latitude, df.start_station_longitude, df.end_station_latitude, df.end_station_longitude))

In [ ]:
df.show()

+----------+---------------+-----------------+------------------+---+--------+-------------+---------------+----------------+---+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|start_time|start time hour|start time minute|start time seconds|_c4|end_time|end_time hour|end_time minute|end_time seconds|_c9|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+----------+---------------+-----------------+------------------+---+--------+-------------+---------------+----------------+---+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------

In [ ]:
start_time = F.concat_ws(":", "start time hour", "start time minute", "start time seconds", "_c4")
end_time = F.concat_ws(":", "end_time hour", "end_time minute", "end_time seconds", "_c9")
df = df.withColumn("starttime", to_timestamp('start_time'))
df = df.withColumn("endtime", to_timestamp('end_time'))

In [ ]:
df.show()


In [ ]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, when, concat_ws, to_timestamp

In [ ]:
df = df.withColumn("duration", when(df.start_time < df.end_time, (df.end_time - df.start_time). cast("double")).otherwise((df.start_time - df.end_time).cast("double")))

In [ ]:
payment_ration = df.groupBy('pyment').count()
display(payment_ration)

pyment,count
app wallet,260061
credit card,259639


In [ ]:
ratio_pyment = df.filter(df.pyment == "app wallet").count() / df.filter(df.pyment == 'credit card').count()
ratio_pyment

1.0016253336363181

In [ ]:
users = df.groupBy(['user_type', 'pyment']).count()
users

user_type,pyment,count
Subscriber,app wallet,204727
Customer,app wallet,55334
Customer,credit card,55136
Subscriber,credit card,204503


In [ ]:
user_pivot = df.groupBy('user_type').pivot('pyment').count()
user_pivot

user_type,app wallet,credit card
Subscriber,204727,204503
Customer,55334,55136
